### seq2seq模型(encoder + attention + decoder)
* CNN
    * 权重共享
        * 平移不变性（ 据偏置归纳）
        * 可并行计算
    * 滑动窗口
        * 局部关联性建模
            * 依靠多层堆积来进行长程建模（感受野）
    * 对相对位置敏感，对绝对位置不敏感
* RNN：依次有序进行建模
    * 对顺序敏感
    * 串行计算耗时
    * 长程建模能力弱
    * 计算复杂度与序列长度呈线性关系
    * 单步计算复杂度不变
    * 对相对位置敏感，对绝对位置敏感
* Transformer
    * 无局部假设
        * 可并行计算
        * 对相对位置不敏感
    * 无有序假设
        * 需要位置编码来反映位置变化对于特征的影响：Positional Embedding
        * 对绝对位置不敏感
    * 任意两字符都可以建模
        * 擅长长短程建模
        * 自注意力机制需要序列长度的平方级别复杂度（缺点）

### Transformer结构

#### Encoder:
* input word embedding
    * 由稀疏的one-hot编码进入一个不带bias的FFN得到一个稠密的连续向量
* positional encoding
    * 通过sin/cos来固定表征
        * 每个位置是确定性的
        * 对于不同的句子，相同位置的距离一致
        * 可以推广到更长的测试句子
    * pe(pos+k)可以写成P(k)的线性组合
    * 通过残差连接使得位置信息流入深层
* multi-head self-attention
    * 使得建模能力更强，表征空间更丰富
    * 由多组Q,K,VB构成，每组单独计算一个attention向量
    * 把每组的attention向量拼起来，并进入一个不带bias的FFN得到最终的向量
* feed-forward network
    * 只考虑每个单独位置进行建模
    * 不同位置参数共享
    * 类似于1*1的 pointwise convolution

#### Decoder
* output word embedding
* multi-head self-attention
* multi-hrad cross-attention
* feed-forward network
* softmax

Transformer难点与实现
* word embedding
* position embedding
* encoder self-attention mask
* intra-attention mask
* decoder self-attention mask
* multi-head self-attention

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
# Word Embedding
# 以序列建模为例
# 考虑source sentence和target sentence

# 构建序列，序列的字符以索引的形式表示
# 源序列和目标序列的长度
batch_size = 2
# 单词表大小
max_num_src_words = 8
max_num_tgt_words = 8
model_dim = 8 # 模型特征大小
# 序列的最大长度
max_src_seq_len = 5
max_tgt_seq_len = 5

src_len = torch.randint(2, 5, (batch_size,)).to(torch.int32)
tgt_len = torch.randint(2, 5, (batch_size,)).to(torch.int32)
print(src_len, tgt_len)

# 以单词索引构成的源句子和目标句子，构建batch
# 需要pad成最大长度，向其中填充0
src_seq = torch.cat([
    torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)), (0, max_src_seq_len - L)), 0) \
        for L in src_len
    ])
tgt_seq = torch.cat([
    torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)), (0, max_tgt_seq_len - L)), 0) \
        for L in tgt_len
    ])
print(src_seq, tgt_seq)

# 构造word embedding
src_embedding_table = nn.Embedding(max_num_src_words + 1, model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words + 1, model_dim)

print(src_embedding_table.weight)

src_embedding = src_embedding_table(src_seq)
tgt_embedding = tgt_embedding_table(tgt_seq)
print('src_seq: ', src_seq)
print('tgt_seq', tgt_seq)
print('src_embedding', src_embedding, src_embedding.shape)
print('tgt_embedding', tgt_embedding, tgt_embedding.shape)

tensor([4, 4], dtype=torch.int32) tensor([4, 4], dtype=torch.int32)
tensor([[4, 4, 4, 6, 0],
        [2, 6, 3, 7, 0]]) tensor([[5, 6, 7, 4, 0],
        [1, 4, 6, 7, 0]])
Parameter containing:
tensor([[-0.3057, -1.1836, -0.0845, -0.4707, -1.9558, -0.6312, -1.6973, -0.8162],
        [-1.6348, -0.5626, -0.3832,  1.1118, -0.6217, -0.3915, -0.0535, -0.3769],
        [-0.4993,  0.3921, -1.3122,  1.0456, -0.9589,  0.4589,  0.1509, -0.5040],
        [ 0.2234,  0.0507, -0.0209,  1.8107,  0.6105,  2.2234, -0.0382,  2.5241],
        [-1.7896, -1.3420,  1.3553,  0.7506, -0.2260, -1.1905,  1.7992, -1.2508],
        [ 1.2360, -1.3867, -0.3615, -1.8234, -0.8902,  0.1038,  0.2267,  0.3105],
        [-0.1237, -0.2180,  1.2855,  0.7839, -0.5290,  1.0634, -0.5195, -1.9405],
        [-0.9626,  0.1429,  1.3647, -0.1821, -0.5914,  1.5330,  1.2365, -0.6320],
        [-0.3460,  0.4774, -0.5588, -0.1905, -3.0375, -1.6223, -1.5261,  0.2128]],
       requires_grad=True)
src_seq:  tensor([[4, 4, 4, 6, 0],
       

In [24]:
# 构造position embedding
max_position_len = 5
pos_matrix = torch.arange(max_position_len).reshape((-1, 1))
index_matrix = torch.pow(10000, torch.arange(0, 8, 2).reshape((1, -1)) / model_dim)
pe_table = torch.zeros(max_position_len, model_dim)
pe_table[:, 0::2] = torch.sin(pos_matrix / index_matrix) # 其余全部交给广播操作
pe_table[:, 1::2] = torch.cos(pos_matrix / index_matrix)
print(pos_matrix)
print(index_matrix)
print(pe_table)

tensor([[0],
        [1],
        [2],
        [3],
        [4]])
tensor([[   1.,   10.,  100., 1000.]])
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
          9.9995e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
          9.9980e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
          9.9955e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01, -6.5364e-01,  3.8942e-01,  9.2106e-01,  3.9989e-02,
          9.9920e-01,  4.0000e-03,  9.9999e-01]])


In [ ]:
pe = nn.Embedding(max_position_len, model_dim)
pe.weight = nn.Parameter(pe_table, requires_grad=False)

src_position = [torch.arange(L) for L in src_len]

print(pe.weight, pe.weight.shape)
print(src_seq.shape)
src_pe = pe(src_seq)

Parameter containing:
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
          9.9995e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
          9.9980e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
          9.9955e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01, -6.5364e-01,  3.8942e-01,  9.2106e-01,  3.9989e-02,
          9.9920e-01,  4.0000e-03,  9.9999e-01]]) torch.Size([5, 8])
torch.Size([2, 5])


IndexError: index out of range in self

In [3]:
# ======================================
# === Pytorch手写Transformer完整代码
# ======================================
# 数据构建
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [ ]:
##############################
# 这里是最开始试的时候使用的，之后任务改为机器翻译的任务
#################################

device = 'cpu'
# device = 'cuda'

# transformer epochs
epochs = 100
# epochs = 1000

# 这里我没有用什么大型的数据集，而是手动输入了两对中文→英语的句子
# 还有每个字的索引也是我手动硬编码上去的，主要是为了降低代码阅读难度
# S: Symbol that shows Start of decoding input
# E: Symbol that shows End of decoding output
# P: Symbol that will fill in blank sequence if current batch data size is short than time steps

# 训练集
sentences = [
    # 中文和英语的单词个数不要求相同
    # enc_input                dec_input           dec_output
    ['我 有 一 个 好 朋 友 P', 'S I have a good friend .', 'I have a good friend . E'],
    ['我 有 零 个 女 朋 友 P', 'S I have zero girl friend .', 'I have zero girl friend . E'],
    ['我 有 一 个 男 朋 友 P', 'S I have a boy friend .', 'I have a boy friend . E']
]

# 测试集（希望transformer能达到的效果）
# 输入："我 有 一 个 女 朋 友"
# 输出："i have a girlfriend"

# 中文和英语的单词要分开建立词库
# Padding Should be Zero

src_vocab = {'P': 0, '我': 1, '有': 2, '一': 3,
             '个': 4, '好': 5, '朋': 6, '友': 7, '零': 8, '女': 9, '男': 10}
src_idx2word = {i: w for i, w in enumerate(src_vocab)}
src_vocab_size = len(src_vocab)

tgt_vocab = {'P': 0, 'I': 1, 'have': 2, 'a': 3, 'good': 4,
             'friend': 5, 'zero': 6, 'girl': 7,  'boy': 8, 'S': 9, 'E': 10, '.': 11}
idx2word = {i: w for i, w in enumerate(tgt_vocab)}
tgt_vocab_size = len(tgt_vocab)

src_len = 8  # （源句子的长度）enc_input max sequence length
tgt_len = 7  # dec_input(=dec_output) max sequence length

# Transformer Parameters
# d_model
    # 是每一层Encoder和Decoder Layer的特征维度，一直保持一致
    # 表示Transformer的语义空间维度，是模型捕捉语义信息的核心特征空间
d_model = 512  # Embedding Size（token embedding和position编码的维度）

# FeedForward dimension (两次线性层中的隐藏层 512->2048->512，线性层是用来做特征提取的）
    # 增强特征表达能力
        # 多头注意力关注的是输入序列中不同位置之间的关系，但是不足以捕捉每个位置的丰富特征
        # FF Layer是使用更高维度的中间层（通常4 * d_model）对每个位置的特征进行非线性变换，从而提升表达能力
    # 加强非线性能力
        # 注意力机制本质上还是线性的，FFN引入非线性激活函数使模型更具有表达力
    # 实现point-wise逐位置特征更新
        # FFN对输入序列的每个位置单独计算
# 当然最后会再接一个projection层，是模型输出部分（特别是Transformer Decoder输出后）的线性映射层
    # 映射到词汇表大小
    # 计算最终的概率分布，完成最终的序列预测任务
    
d_ff = 2048
d_k = d_v = 64  # dimension of K(=Q), V（Q和K的维度需要相同，这里为了方便让K=V）
n_layers = 6  # number of Encoder of Decoder Layer（Block的个数）
n_heads = 8  # number of heads in Multi-Head Attention（有几套头）


# ==============================================================================================
# 数据构建

def make_data(sentences):
    """把单词序列转换为数字序列"""
    enc_inputs, dec_inputs, dec_outputs = [], [], []
    for i in range(len(sentences)):
        enc_input = [[src_vocab[n] for n in sentences[i][0].split()]]
        dec_input = [[tgt_vocab[n] for n in sentences[i][1].split()]]
        dec_output = [[tgt_vocab[n] for n in sentences[i][2].split()]]

        #[[1, 2, 3, 4, 5, 6, 7, 0], [1, 2, 8, 4, 9, 6, 7, 0], [1, 2, 3, 4, 10, 6, 7, 0]]
        enc_inputs.extend(enc_input) # 使用extend是因为enc_input等是两层的列表
        # print(enc_inputs)
        #[[9, 1, 2, 3, 4, 5, 11], [9, 1, 2, 6, 7, 5, 11], [9, 1, 2, 3, 8, 5, 11]]
        dec_inputs.extend(dec_input)
        #[[1, 2, 3, 4, 5, 11, 10], [1, 2, 6, 7, 5, 11, 10], [1, 2, 3, 8, 5, 11, 10]]
        dec_outputs.extend(dec_output)
    return torch.LongTensor(enc_inputs), torch.LongTensor(dec_inputs), torch.LongTensor(dec_outputs) # longtensor主要特指int64


enc_inputs, dec_inputs, dec_outputs = make_data(sentences)


class MyDataSet(Data.Dataset):
    """自定义DataLoader"""

    def __init__(self, enc_inputs, dec_inputs, dec_outputs):
        super(MyDataSet, self).__init__()
        self.enc_inputs = enc_inputs
        self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs

    def __len__(self):
        return self.enc_inputs.shape[0]

    def __getitem__(self, idx):
        return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]


loader = Data.DataLoader(
    MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True)


In [ ]:
import re
from typing import Tuple, Dict

def build_vocab(sentences) -> Tuple[Dict[str, int], Dict[int, str], int]:
    '''构建词汇表，返回最大长度'''
    max_len = 1
    word2idx = {'P': 0, 'S': 1, 'E': 2}
    idx2word = {0: 'P', 2: 'S', 3: 'E'}
    
    for sentence in sentences:
        # 使用正则表达式分词，确保标点符号作为独立的词
        words = re.findall(r'\w+|[^\s\w]', sentence)
        if len(words) > max_len:
            max_len = len(words)
        for word in words:
            if word not in word2idx:
                idx = len(word2idx)
                word2idx[word] = idx
                idx2word[idx] = word
    
    return word2idx, idx2word, max_len + 1

def make_data(src_file, tgt_file, src_vocab, tgt_vocab, max_src_len, max_tgt_len):
    """从文件读取数据并转换为模型所需格式"""
    enc_inputs, dec_inputs, dec_outputs = [], [], []
    with open(src_file, 'r', encoding='utf-8') as f_src, open(tgt_file, 'r', encoding='utf-8') as f_tgt:
        for src_line, tgt_line in zip(f_src, f_tgt):
            # 使用正则表达式分词，确保标点符号被单独处理
            src_tokens = re.findall(r'\w+|[^\s\w]', src_line.strip())
            tgt_tokens = re.findall(r'\w+|[^\s\w]', tgt_line.strip())

            # 添加开始（S）和结束（E）符号
            dec_input_tokens = ['S'] + tgt_tokens
            dec_output_tokens = tgt_tokens + ['E']

            # 截断或填充
            src_seq = [src_vocab.get(word, 0) for word in src_tokens[:max_src_len]]
            tgt_input_seq = [tgt_vocab.get(word, 0) for word in dec_input_tokens[:max_tgt_len]]
            tgt_output_seq = [tgt_vocab.get(word, 0) for word in dec_output_tokens[:max_tgt_len]]

            src_seq += [0] * (max_src_len - len(src_seq))  # Padding
            tgt_input_seq += [0] * (max_tgt_len - len(tgt_input_seq))
            tgt_output_seq += [0] * (max_tgt_len - len(tgt_output_seq))

            enc_inputs.append(src_seq)
            dec_inputs.append(tgt_input_seq)
            dec_outputs.append(tgt_output_seq)
            
    return torch.LongTensor(enc_inputs), torch.LongTensor(dec_inputs), torch.LongTensor(dec_outputs)


class MyDataSet(Data.Dataset):
    """自定义DataLoader"""

    def __init__(self, enc_inputs, dec_inputs, dec_outputs):
        super(MyDataSet, self).__init__()
        self.enc_inputs = enc_inputs
        self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs

    def __len__(self):
        return self.enc_inputs.shape[0]

    def __getitem__(self, idx):
        return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]
       

In [5]:
src_file = '/home/shenc/Desktop/wmt14/europarl-v7.de-en.de'
tgt_file = '/home/shenc/Desktop/wmt14/europarl-v7.de-en.en'

# 从文件中读数据
with open(src_file, 'r', encoding='utf-8') as f_src, open(tgt_file, 'r', encoding='utf-8') as f_tgt:
    src_sentences = f_src.readlines()
    tgt_sentences = f_tgt.readlines()

# 从这里获得最大序列长度，方便以后
src_vocab, src_idx2word, max_len_src = build_vocab(src_sentences)
tgt_vocab, tgt_idx2word, max_len_tgt = build_vocab(tgt_sentences)

In [7]:
src_vocab

{'P': 0,
 'Wiederaufnahme': 1,
 'der': 2,
 'Sitzungsperiode': 3,
 'Ich': 4,
 'erkläre': 5,
 'die': 6,
 'am': 7,
 'Freitag,': 8,
 'dem': 9,
 '17.': 10,
 'Dezember': 11,
 'unterbrochene': 12,
 'des': 13,
 'Europäischen': 14,
 'Parlaments': 15,
 'für': 16,
 'wiederaufgenommen,': 17,
 'wünsche': 18,
 'Ihnen': 19,
 'nochmals': 20,
 'alles': 21,
 'Gute': 22,
 'zum': 23,
 'Jahreswechsel': 24,
 'und': 25,
 'hoffe,': 26,
 'daß': 27,
 'Sie': 28,
 'schöne': 29,
 'Ferien': 30,
 'hatten.': 31,
 'Wie': 32,
 'feststellen': 33,
 'konnten,': 34,
 'ist': 35,
 'gefürchtete': 36,
 '"Millenium-Bug': 37,
 '"': 38,
 'nicht': 39,
 'eingetreten.': 40,
 'Doch': 41,
 'sind': 42,
 'Bürger': 43,
 'einiger': 44,
 'unserer': 45,
 'Mitgliedstaaten': 46,
 'Opfer': 47,
 'von': 48,
 'schrecklichen': 49,
 'Naturkatastrophen': 50,
 'geworden.': 51,
 'Im': 52,
 'Parlament': 53,
 'besteht': 54,
 'Wunsch': 55,
 'nach': 56,
 'einer': 57,
 'Aussprache': 58,
 'im': 59,
 'Verlauf': 60,
 'dieser': 61,
 'in': 62,
 'den': 63,
 'näc

In [ ]:
# S开始     E结束       P空字符标记
# 训练集，是使用两个文件表示翻译前后的内容
src_file = '/home/shenc/Desktop/wmt14/europarl-v7.de-en.de'
tgt_file = '/home/shenc/Desktop/wmt14/europarl-v7.de-en.en'

# 从文件中读数据
with open(src_file, 'r', encoding='utf-8') as f_src, open(tgt_file, 'r', encoding='utf-8') as f_tgt:
    src_sentences = f_src.readlines()
    tgt_sentences = f_tgt.readlines()

# 从这里获得最大序列长度，方便以后
src_vocab, src_idx2word, max_len_src = build_vocab(src_sentences)
tgt_vocab, tgt_idx2word, max_len_tgt = build_vocab(tgt_sentences)

enc_inputs, dec_inputs, dec_outputs = make_data(
    src_file, tgt_file, src_vocab, tgt_vocab, max_len_src, max_len_tgt
)

src_len = max_len_src
tgt_len = max_len_tgt

src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs = 1000

d_model = 512  # Embedding Size（token embedding和position编码的维度）

d_ff = 2048
d_k = d_v = 64  # dimension of K(=Q), V（Q和K的维度需要相同，这里为了方便让K=V）
n_layers = 6  # number of Encoder of Decoder Layer（Block的个数）
n_heads = 8  # 有几个头hhh


loader = Data.DataLoader(
    MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True
)


In [4]:

# ====================================================================================================
# Transformer模型

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        '''
        初始化位置编码类
        d_model:    模型的维度，即每个词的嵌入维度
        dropout:    防止过拟合的dropout
        max_len:    最大序列长度，用于生成位置编码
        '''
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model) # 用于存储位置编码
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(
            0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # 偶数维度使用正弦，奇数维使用余弦
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # 添加一个batch维度并转置
        pe = pe.unsqueeze(0).transpose(0, 1)
        
        # 将位置编码存储为不可训练的缓冲区
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        将位置编码添加到张量x中
        x: [seq_len, batch_size, d_model]
        加入位置编码并dropout之后的张量
        """
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


def get_attn_pad_mask(seq_q, seq_k):
    """
    创建 Padding Mask ，用于屏蔽序列中的填充（PAD）位置
    这里的q,k表示的是两个序列（跟注意力机制的q,k没有关系），
    
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    
    返回 
        pad_attn_mask :形状为[batch_size, len_q, len_k]的掩码, True表示被掩盖的位置
    """
    batch_size, len_q = seq_q.size()  # 这个seq_q只是用来expand维度的
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    # 例如:seq_k = [[1,2,3,4,0], [1,2,3,5,0]]
    # [batch_size, 1, len_k], True is masked
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1) # 在某维插入新维度
    # 还可以写作pad_attn_mask = (seq_k.data == 0).unsqueeze(1)
    # [batch_size, len_q, len_k] 构成一个立方体(batch_size个这样的矩阵)
    return pad_attn_mask.expand(batch_size, len_q, len_k)


def get_attn_subsequence_mask(seq):
    """
    seq: [batch_size, tgt_len]
    """
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    # attn_shape: [batch_size, tgt_len, tgt_len]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1)  # 生成一个上三角矩阵
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()
    return subsequence_mask  # [batch_size, tgt_len, tgt_len]


# ==========================================================================================
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        """
        Q: Query [batch_size, n_heads, len_q, d_k]
        K: Key [batch_size, n_heads, len_k, d_k]
        V: Value [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        说明：在encoder-decoder的Attention层中len_q(q1,..qt)和len_k(k1,...km)可能不同
        """
        
        # 计算注意力分数并缩放
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)  
        # scores : [batch_size, n_heads, len_q, len_k]
        
        # mask矩阵填充scores（用-1e9填充scores中与attn_mask中值为1位置相对应的元素），使用掩码掩盖不需要的注意力
        # Fills elements of self tensor with value where mask is True.
        scores.masked_fill_(attn_mask, -1e9)

        attn = nn.Softmax(dim=-1)(scores)  # 对最后一个维度(v)做softmax
        # scores : [batch_size, n_heads, len_q, len_k] * V: [batch_size, n_heads, len_v(=len_k), d_v]
        # context: [batch_size, n_heads, len_q, d_v]
        context = torch.matmul(attn, V)
        # context：[[z1,z2,...],[...]]向量, attn注意力稀疏矩阵（用于可视化的）
        return context, attn


class MultiHeadAttention(nn.Module):
    """
    实现多头注意力机制，用于三种场景：
        Encoder的Self-Attention
        Decoder的Masked Self-Attention
        Encoder-Decoder的Attention
    输入：seq_len x d_model
    输出：seq_len x d_model
    """

    def __init__(self):
        """
        实现多头注意力机制。

        参数:
        input_Q: query，形状为 [batch_size, len_q, d_model]
        input_K: key，形状为 [batch_size, len_k, d_model]
        input_V: value，形状为 [batch_size, len_v, d_model]
        attn_mask: 注意力掩码，形状为 [batch_size, len_q, len_k]

        返回:
        output: 输出张量，形状为 [batch_size, len_q, d_model]
        attn: 注意力权重
        """
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)  # q,k必须维度相同，不然无法做点积
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        
        # 多头注意力的输出需要与输入维度一致（不然怎么加residual呢）
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)

    def forward(self, input_Q, input_K, input_V, attn_mask):
        """
        计算多头注意力的输出和注意力权重
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        """
        
        residual, batch_size = input_Q, input_Q.size(0)
        # 下面的多头的参数矩阵是放在一起做线性变换的，然后再拆成多个头，这是工程实现的技巧
        # B: batch_size, S:seq_len, D: dim
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, Head, W) -trans-> (B, Head, S, W)
        #           线性变换               拆成多头

        # 1. 将输入的Q、K、V投影到多头子空间
        # Q: [batch_size, n_heads, len_q, d_k]
        Q = self.W_Q(input_Q).view(batch_size, -1,
                                   n_heads, d_k).transpose(1, 2)
        # K: [batch_size, n_heads, len_k, d_k] # K和V的长度一定相同，维度可以不同
        K = self.W_K(input_K).view(batch_size, -1,
                                   n_heads, d_k).transpose(1, 2)
        # V: [batch_size, n_heads, len_v(=len_k), d_v]
        V = self.W_V(input_V).view(batch_size, -1,
                                   n_heads, d_v).transpose(1, 2)

        # 2. 扩展注意力掩码的维度以适应多头机制
        # attn_mask: [batch_size, seq_len, seq_len] -> [batch_size, n_heads, seq_len, seq_len]
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)

        # 3. 计算注意力
        # context: [batch_size, n_heads, len_q, d_v]
        # attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        
        # 4. 将不同头的输出向量拼接在一起
        # context: [batch_size, n_heads, len_q, d_v] -> [batch_size, len_q, n_heads * d_v]
        context = context.transpose(1, 2).reshape(
            batch_size, -1, n_heads * d_v)

        # 5. 使用全连接层，将维度映射会d_model
        output = self.fc(context)  # [batch_size, len_q, d_model]
        
        # 6. 残差连接和Layer Norm
        return nn.LayerNorm(d_model).to(device)(output + residual), attn


# Pytorch中的Linear只会对最后一维操作，所以正好是我们希望的每个位置用同一个全连接网络
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )

    def forward(self, inputs):
        """
        inputs: [batch_size, seq_len, d_model]
        """
        residual = inputs
        output = self.fc(inputs)
        # [batch_size, seq_len, d_model]
        return nn.LayerNorm(d_model).to(device)(output + residual)


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        # 这个是多头子注意力模块，用于处理 Encoder 输入的自身关系
        self.enc_self_attn = MultiHeadAttention()
        # 逐位置的前馈网络，进一步处理多头注意力的输出
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        """
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]  mask矩阵(pad mask or sequence mask)
        返回：
            enc_outputs: [batch_size, src_len, d_model]
            attn: [batch_size, n_heads, src_len, src_len]
        """
        # 自注意力模块：
        # 输入 enc_inputs 作为 Q, K, V（源序列内部的关系）
        # 得到处理后的特征和注意力权重
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)  
        
        # 前馈网络
        # 对多头注意力模块的输出进行逐位置的特征更新，保持形状不变
        enc_outputs = self.pos_ffn(enc_outputs)
        
        # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn


class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        # 自注意力模块
        self.dec_self_attn = MultiHeadAttention()
        # 编码器-解码器注意力模块，用于关注编码器的输出特征
        self.dec_enc_attn = MultiHeadAttention()
        # 前馈网络，用于对特征进一步处理
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        """
        - dec_inputs: [batch_size, tgt_len, d_model]
        - enc_outputs: [batch_size, src_len, d_model]
            编码器的输入序列，用作接码器的参考
        - dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
            自注意力掩码，用来屏蔽未来时间步（防止泄漏未来信息）
        - dec_enc_attn_mask: [batch_size, tgt_len, src_len]
            编码器-解码器注意力的掩码，用于屏蔽编码器的填充位置
            
        返回：
            解码器输出序列：[batch_size, tgt_len, d_model], 
            自注意力权重：[batch_size, n_heads, tgt_len, tgt_len]
            编码器，解码器注意力权重
        """
        # 1. 解码器自注意力机制
        #   处理解码器自己的输入序列，掩盖未来时间步的信息（decoder需要按序生成）
        dec_outputs, dec_self_attn = self.dec_self_attn(
            dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)  # 这里的Q,K,V全是Decoder自己的输入
        # dec_outputs: [batch_size, tgt_len, d_model], 
        # dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
        # 2. 编码器-解码器注意力机制
        #   解码器输出作为Q，编码器输出作为K和V用于解码器关注源序列的信息
        dec_outputs, dec_enc_attn = self.dec_enc_attn(
            dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)  # Attention层的Q(来自decoder) 和 K,V(来自encoder)

        # 3. 前馈网络
        #   对解码器的输出序列逐位置地曾请特征表达能力
        dec_outputs = self.pos_ffn(dec_outputs)
        # 返回解码器的输出以及两个注意力权重（用与可视化）
        return dec_outputs, dec_self_attn, dec_enc_attn

####################################################
# 编码器的任务是将源序列映射到一个固定维度的表示
#   Encoder的自注意力机制处理的是源序列的所有信息，允许每个位置都能够访问其他位置的信息
#   由于没有生成序列的时间顺序问题，编码器的输入序列可以并行处理，且每个token之间的依赖关系可以是全局的
#   编码器不用mask来演示信息流动，所有位置都彼此关注
# 解码器的任务是根据编码器的输出（源序列的表示）生成目标序列
#   Decoder的子注意力需要掩码，以确保模型不会看到未来时间步的内容
######################################################

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        # token Embedding 将输入的词汇表索引转换为词向量
        self.src_emb = nn.Embedding(src_vocab_size, d_model) # 输入的词嵌入层，词汇表大小为src_vocab_size，d_model为词向量维度
        # 位置编码（Transformer中位置编码时固定的，不需要学习）
        self.pos_emb = PositionalEncoding(d_model) 
        # 定义多个EncoderLayer：每一层都由自注意力机制和前馈网络组成
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        """
        enc_inputs: [batch_size, src_len]
        """
        enc_outputs = self.src_emb(enc_inputs)  # 词嵌入的结果：[batch_size, src_len, d_model]
        
        # 加上位置编码，位置编码提供每个词在序列中的位置信息
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1)  # [batch_size, src_len, d_model]
        
        # Encoder输入序列的pad mask矩阵，避免自注意力计算的时候让padding部分参与计算
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)  # [batch_size, src_len, src_len]
        
        # 博阿村每一层计算的注意力矩阵（可视化时使用，如画热力图来看各个词之间的关系）
        enc_self_attns = []
        for layer in self.layers:  # 遍历每一层的Encoder Layer
            # enc_outputs: [batch_size, src_len, d_model]
            # enc_self_attn: [batch_size, n_heads, src_len, src_len]
            # 传入的enc_outputs其实是input，传入mask矩阵是因为你要做self attention
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)  
            enc_self_attns.append(enc_self_attn)  # 保存注意力矩阵做可视化
        return enc_outputs, enc_self_attns


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        # 目标词嵌入层：将目标词汇索引转换为词向量
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)  # 目标词嵌入
        # 位置编码：和Encoder是一样的，位置编码不需要学习
        self.pos_emb = PositionalEncoding(d_model)
        # 定义多个 DecoderLayerL每一层由自注意力机制和Encoder-Decoder注意力机制组成
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        """
        dec_inputs: [batch_size, tgt_len]
        enc_inputs: [batch_size, src_len]
        enc_outputs: [batch_size, src_len, d_model]   # 用在Encoder-Decoder Attention层
        """
        dec_outputs = self.tgt_emb(dec_inputs)  # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1).to(device)  # [batch_size, tgt_len, d_model]
        
        # Decoder输入序列的pad mask矩阵（这个例子中decoder是没有加pad的，实际应用中都是有pad填充的）
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).to(device)  # [batch_size, tgt_len, tgt_len]
        
        # Masked Self_Attention Mask：当前时刻是看不到未来的信息的
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs).to(device)  # [batch_size, tgt_len, tgt_len]

        # Decoder中把两种mask矩阵相加（既屏蔽了padding，也屏蔽了未来时刻的信息）
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask),0).to(device)  
        # [batch_size, tgt_len, tgt_len]; torch.gt比较两个矩阵的元素，大于则返回1，否则返回0

        # 这个mask主要用于encoder-decoder attention层
        # get_attn_pad_mask主要是enc_inputs的pad mask矩阵(因为enc是处理K,V的，求Attention时是用v1,v2,..vm去加权的，要把pad对应的v_i的相关系数设为0，这样注意力就不会关注pad向量)
        #                       dec_inputs只是提供expand的size的
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)  # [batc_size, tgt_len, src_len]

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model]
            # dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
            # dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn, dec_enc_attn = layer(
                dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        # dec_outputs: [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attns, dec_enc_attns


class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        # 初始化各部分组件
        self.encoder = Encoder().to(device)
        self.decoder = Decoder().to(device)
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False).to(device)

    def forward(self, enc_inputs, dec_inputs):
        """
        Transformers前向传播，接受源语言和目标语言序列作为输入
        
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, tgt_len]
        """
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # enc_outputs: [batch_size, src_len, d_model]
        # enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        # 经过Encoder网络后，得到的输出还是[batch_size, src_len, d_model]
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        # dec_outputs: [batch_size, tgt_len, d_model]
        # dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len]
        # dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        # dec_outputs: [batch_size, tgt_len, d_model] -> dec_logits: [batch_size, tgt_len, tgt_vocab_size]
        dec_logits = self.projection(dec_outputs) # 将解码器的输出从d_model映射到词汇表大小
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns


model = Transformer().to(device)
# 这里的损失函数里面设置了一个参数 ignore_index=0，因为 "pad" 这个单词的索引为 0，这样设置以后，就不会计算 "pad" 的损失（因为本来 "pad" 也没有意义，不需要计算）
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)  # 用adam的话效果不好

# ====================================================================================================
# 训练循环
for epoch in range(epochs):
    for enc_inputs, dec_inputs, dec_outputs in loader:
        """
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, tgt_len]
        dec_outputs: [batch_size, tgt_len]
        """
        enc_inputs, dec_inputs, dec_outputs = enc_inputs.to(
            device), dec_inputs.to(device), dec_outputs.to(device)
        # outputs: [batch_size * tgt_len, tgt_vocab_size]
        outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
        # dec_outputs.view(-1):[batch_size * tgt_len * tgt_vocab_size]
        loss = criterion(outputs, dec_outputs.view(-1))
        print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def greedy_decoder(model, enc_input, start_symbol):
    """贪心编码
    贪心解码是当K=1时的束搜索（Beam Search）。在推理阶段，由于我们不知道目标序列的输入，
    因此我们通过逐步生成目标序列的每个单词，并将其逐步输入到Transformer模型中进行预测。
    Starting Reference: http://nlp.seas.harvard.edu/2018/04/03/attention.html#greedy-decoding
    :param model: Transformer Model
    :param enc_input: The encoder input
    :param start_symbol: The start symbol. In this example it is 'S' which corresponds to index 4
    :return: The target input
    """
    # 使用Encoder处理源语言输入，得到Encoder的输出以及自注意力权重
    enc_outputs, enc_self_attns = model.encoder(enc_input)
    
    # 初始化一个空的tensor: tensor([], size=(1, 0), dtype=torch.int64)
    # 此张量表示解码器尚未生成任何单词
    dec_input = torch.zeros(1, 0).type_as(enc_input.data)
    terminal = False # 终止标志
    next_symbol = start_symbol # 下一个要生成的单词，初始化为起始符号
    
    while not terminal:
        # 循环，直到生成结束符
        # 预测阶段：dec_input序列会一点点变长（每次添加一个新预测出来的单词）
        dec_input = torch.cat(
            [dec_input.to(device), torch.tensor([[next_symbol]], dtype=enc_input.dtype).to(device)], -1)
        # 使用解码器处理当前解码器输入和编码器输出，得到解码器的输出
        dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)
        # 使用Decoder的输出通过Proj层映射到目标词汇表大小，得到每个单词的概率分布
        projected = model.projection(dec_outputs)
        
        # 获取当前时间步的预测概率（即最大值所在的位置，代表预测的单词）
        # projected.squeeze(0): 移除batch维度，获取形状为[tgt_len, tgt_vocab_size]的张量
        # max(dim=-1)是为了取出每个位置的最大概率，并返回其对应的索引
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        
        # 增量更新（我们希望重复单词预测结果是一样的）
        # 我们在预测是会选择性忽略重复的预测的词，只摘取最新预测的单词拼接到输入序列中
        # 拿出当前预测的单词(数字)。我们用x'_t对应的输出z_t去预测下一个单词的概率，不用z_1,z_2..z_{t-1}
        next_word = prob.data[-1]
        next_symbol = next_word
        
        if next_symbol == tgt_vocab["E"]:
            terminal = True
        # print(next_word)
    # greedy_dec_predict = torch.cat(
    #     [dec_input.to(device), torch.tensor([[next_symbol]], dtype=enc_input.dtype).to(device)],
    #     -1)
    # 返回解码器的预测序列，去掉了开始符号部分（即dec_input[:, 1:]）
    # 因为解码器输入时已加上了起始符号，现在要去除起始符号
    greedy_dec_predict = dec_input[:, 1:]
    return greedy_dec_predict

Epoch: 0001 loss = 2.793939
Epoch: 0001 loss = 2.622025
Epoch: 0002 loss = 2.453482
Epoch: 0002 loss = 2.383991
Epoch: 0003 loss = 2.218930
Epoch: 0003 loss = 1.631452
Epoch: 0004 loss = 1.645802
Epoch: 0004 loss = 1.601075
Epoch: 0005 loss = 1.179267
Epoch: 0005 loss = 1.340614
Epoch: 0006 loss = 0.965044
Epoch: 0006 loss = 0.678392
Epoch: 0007 loss = 0.552932
Epoch: 0007 loss = 0.631039
Epoch: 0008 loss = 0.398819
Epoch: 0008 loss = 0.395581
Epoch: 0009 loss = 0.269114
Epoch: 0009 loss = 0.259883
Epoch: 0010 loss = 0.202923
Epoch: 0010 loss = 0.211473
Epoch: 0011 loss = 0.218503
Epoch: 0011 loss = 0.202608
Epoch: 0012 loss = 0.121742
Epoch: 0012 loss = 0.268149
Epoch: 0013 loss = 0.164403
Epoch: 0013 loss = 0.083710
Epoch: 0014 loss = 0.153977
Epoch: 0014 loss = 0.056936
Epoch: 0015 loss = 0.137254
Epoch: 0015 loss = 0.079034
Epoch: 0016 loss = 0.066254
Epoch: 0016 loss = 0.152756
Epoch: 0017 loss = 0.090989
Epoch: 0017 loss = 0.131057
Epoch: 0018 loss = 0.127934
Epoch: 0018 loss = 0

In [7]:
# 预测阶段
# 测试集
sentences = [
    # enc_input                dec_input           dec_output
    ['我 有 零 个 女 朋 友 P', '', ''],
]

enc_inputs, dec_inputs, dec_outputs = make_data(sentences)
test_loader = Data.DataLoader(
    MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True)
enc_inputs, _, _ = next(iter(test_loader))

print()
print("="*30)
print("利用训练好的Transformer模型将中文句子'我 有 零 个 女 朋 友' 翻译成英文句子: ")
for i in range(len(enc_inputs)):
    greedy_dec_predict = greedy_decoder(model, enc_inputs[i].view(
        1, -1).to(device), start_symbol=tgt_vocab["S"])
    print(enc_inputs[i], '->', greedy_dec_predict.squeeze())
    print([src_idx2word[t.item()] for t in enc_inputs[i]], '->',
          [idx2word[n.item()] for n in greedy_dec_predict.squeeze()])


利用训练好的Transformer模型将中文句子'我 有 零 个 女 朋 友' 翻译成英文句子: 
tensor([1, 2, 8, 4, 9, 6, 7, 0]) -> tensor([ 1,  2,  6,  7,  5, 11])
['我', '有', '零', '个', '女', '朋', '友', 'P'] -> ['I', 'have', 'zero', 'girl', 'friend', '.']
